In [13]:
import sqlite3
from tkinter import *
from tkinter import messagebox
from tkinter import ttk
import datetime
import os

# PDF generation (reportlab). If not available, we handle gracefully.
try:
    from reportlab.lib.pagesizes import A4
    from reportlab.pdfgen import canvas
    PDF_AVAILABLE = True
except Exception:
    PDF_AVAILABLE = False

# ------------- Database -------------
DB_FILE = "inventory.db"
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    category TEXT NOT NULL,
    qty INTEGER NOT NULL,
    price REAL NOT NULL,
    country TEXT NOT NULL,
    created_at TEXT NOT NULL
)
""")
conn.commit()

# ---------- Helper Functions ----------
def numeric_or_error(value, field_name):
    try:
        if field_name == "qty":
            v = int(value)
            if v < 0:
                raise ValueError
            return v
        else:  # price
            v = float(value)
            if v < 0:
                raise ValueError
            return v
    except Exception:
        raise ValueError(f"Field '{field_name}' must be a non-negative number.")

# ---------- Core DB Functions ----------
def add_item():
    name = name_entry.get().strip()
    category = category_var.get().strip()
    qty = qty_entry.get().strip()
    price = price_entry.get().strip()
    country = country_entry.get().strip()

    if not name or not category or not qty or not price or not country:
        messagebox.showerror("Error", "Please fill all fields")
        return

    try:
        qty_v = numeric_or_error(qty, "qty")
        price_v = numeric_or_error(price, "price")
    except ValueError as e:
        messagebox.showerror("Invalid input", str(e))
        return

    created_at = datetime.datetime.now().isoformat(timespec='seconds')
    cursor.execute(
        "INSERT INTO inventory (name, category, qty, price, country, created_at) VALUES (?, ?, ?, ?, ?, ?)",
        (name, category, qty_v, price_v, country, created_at)
    )
    conn.commit()
    show_items()
    clear_entries()
    messagebox.showinfo("Success", "Item Added Successfully")

def show_items(rows=None):
    for row in tree.get_children():
        tree.delete(row)

    if rows is None:
        cursor.execute("SELECT * FROM inventory")
        rows = cursor.fetchall()

    for row in rows:
        tree.insert("", END, values=row)

def delete_item():
    selected = tree.focus()
    if not selected:
        messagebox.showerror("Error", "Select an item")
        return

    data = tree.item(selected)
    item_id = data["values"][0]

    if messagebox.askyesno("Confirm Delete", f"Delete item ID {item_id}?"):
        cursor.execute("DELETE FROM inventory WHERE id=?", (item_id,))
        conn.commit()
        show_items()
        messagebox.showinfo("Deleted", "Item Deleted Successfully")

def update_item():
    selected = tree.focus()
    if not selected:
        messagebox.showerror("Error", "Select an item to update")
        return

    data = tree.item(selected)
    item_id = data["values"][0]

    name = name_entry.get().strip()
    category = category_var.get().strip()
    qty = qty_entry.get().strip()
    price = price_entry.get().strip()
    country = country_entry.get().strip()

    if not name or not category or not qty or not price or not country:
        messagebox.showerror("Error", "Please fill all fields")
        return

    try:
        qty_v = numeric_or_error(qty, "qty")
        price_v = numeric_or_error(price, "price")
    except ValueError as e:
        messagebox.showerror("Invalid input", str(e))
        return

    cursor.execute(
        "UPDATE inventory SET name=?, category=?, qty=?, price=?, country=? WHERE id=?",
        (name, category, qty_v, price_v, country, item_id)
    )
    conn.commit()
    show_items()
    clear_entries()
    messagebox.showinfo("Updated", "Item Updated Successfully")

def fill_entries(event):
    selected = tree.focus()
    if not selected:
        return
    data = tree.item(selected)
    values = data["values"]

    name_entry.delete(0, END)
    category_var.set(values[2])
    qty_entry.delete(0, END)
    price_entry.delete(0, END)
    country_entry.delete(0, END)

    name_entry.insert(0, values[1])
    qty_entry.insert(0, values[3])
    price_entry.insert(0, values[4])
    country_entry.insert(0, values[5])

def clear_entries():
    name_entry.delete(0, END)
    qty_entry.delete(0, END)
    price_entry.delete(0, END)
    country_entry.delete(0, END)
    category_var.set(categories[0])

# ---------- Search ----------
def search_items():
    q_id = search_id_entry.get().strip()
    q_name = search_name_entry.get().strip()

    query = "SELECT * FROM inventory WHERE 1=1"
    params = []
    if q_id:
        query += " AND id = ?"
        params.append(q_id)
    if q_name:
        query += " AND name LIKE ?"
        params.append(f"%{q_name}%")

    cursor.execute(query, params)
    rows = cursor.fetchall()
    show_items(rows)

def reset_search():
    search_id_entry.delete(0, END)
    search_name_entry.delete(0, END)
    show_items()

# ---------- Report Screen & PDF ----------
def open_report_screen():
    rep = Toplevel(root)
    rep.title("Inventory Report")
    rep.geometry("700x500")
    rep.configure(bg="#F2EFE9")

    title = Label(rep, text="Inventory Report", font=("Arial", 18, "bold"), bg="#F2EFE9", fg="#4B3A2F")
    title.pack(pady=10)

    summary_frame = Frame(rep, bg="#F2EFE9")
    summary_frame.pack(fill=X, padx=10)

    cursor.execute("SELECT COUNT(*), SUM(qty), SUM(qty*price) FROM inventory")
    total_count, total_qty, total_value = cursor.fetchone()
    total_count = total_count or 0
    total_qty = total_qty or 0
    total_value = total_value or 0.0

    Label(summary_frame, text=f"Total rows: {total_count}", bg="#F2EFE9", fg="#333").grid(row=0, column=0, sticky=W, padx=5)
    Label(summary_frame, text=f"Total quantity (units): {total_qty}", bg="#F2EFE9", fg="#333").grid(row=0, column=1, sticky=W, padx=5)
    Label(summary_frame, text=f"Total inventory value: {total_value:.2f}", bg="#F2EFE9", fg="#333").grid(row=0, column=2, sticky=W, padx=5)

    Label(rep, text="Breakdown by Category", font=("Arial", 12, "bold"), bg="#F2EFE9").pack(pady=(10,0))
    breakdown_frame = Frame(rep, bg="#F2EFE9")
    breakdown_frame.pack(fill=BOTH, expand=False, padx=10)

    bc_tree = ttk.Treeview(breakdown_frame, columns=("Category", "Count", "TotalQty", "TotalValue"), show="headings", height=6)
    bc_tree.heading("Category", text="Category")
    bc_tree.heading("Count", text="Number of Items")
    bc_tree.heading("TotalQty", text="Total Qty")
    bc_tree.heading("TotalValue", text="Total Value")
    bc_tree.pack(fill=X)

    cursor.execute("SELECT category, COUNT(*), SUM(qty), SUM(qty*price) FROM inventory GROUP BY category")
    for row in cursor.fetchall():
        cat, cnt, totq, totv = row
        bc_tree.insert("", END, values=(cat, cnt or 0, totq or 0, f"{(totv or 0.0):.2f}"))

    Label(rep, text="Full Items Preview", font=("Arial", 12, "bold"), bg="#F2EFE9").pack(pady=(10,0))
    preview_frame = Frame(rep, bg="#F2EFE9")
    preview_frame.pack(fill=BOTH, expand=True, padx=10, pady=(0,10))

    preview_tv = ttk.Treeview(preview_frame, columns=("ID","Name","Category","Qty","Price","Country","Created"), show="headings")
    for col, cap in [("ID","ID"),("Name","Name"),("Category","Category"),("Qty","Qty"),("Price","Price"),("Country","Country"),("Created","Created At")]:
        preview_tv.heading(col, text=cap)
    preview_tv.pack(fill=BOTH, expand=True)

    cursor.execute("SELECT * FROM inventory")
    for r in cursor.fetchall():
        preview_tv.insert("", END, values=r)

    btn_frame = Frame(rep, bg="#F2EFE9")
    btn_frame.pack(pady=8)
    Button(btn_frame, text="Generate PDF Report", command=lambda: generate_report_pdf(rep), bg="#8C6A43", fg="white", width=18).pack(side=LEFT, padx=6)
    Button(btn_frame, text="Close", command=rep.destroy, bg="#6F5E53", fg="white", width=12).pack(side=LEFT, padx=6)

def generate_report_pdf(parent_window=None):
    if not PDF_AVAILABLE:
        messagebox.showwarning("PDF library missing",
                               "Generating PDF requires the 'reportlab' package.\n\nInstall it with:\n\npip install reportlab")
        return

    cursor.execute("SELECT * FROM inventory")
    rows = cursor.fetchall()

    if not rows:
        messagebox.showinfo("No data", "No items to include in the report.")
        return

    fname = f"inventory_report_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
    c = canvas.Canvas(fname, pagesize=A4)
    width, height = A4
    margin = 40
    y = height - margin

    c.setFont("Helvetica-Bold", 16)
    c.drawString(margin, y, "Inventory Report")
    y -= 24
    c.setFont("Helvetica", 9)
    c.drawString(margin, y, f"Generated: {datetime.datetime.now().isoformat(sep=' ', timespec='seconds')}")
    y -= 20

    c.setFont("Helvetica-Bold", 10)
    headers = ["ID", "Name", "Category", "Qty", "Price", "Country", "Created At"]
    col_x = [margin, margin+40, margin+240, margin+360, margin+420, margin+480, margin+560]
    for i, h in enumerate(headers):
        c.drawString(col_x[i], y, h)
    y -= 14
    c.line(margin, y, width-margin, y)
    y -= 12
    c.setFont("Helvetica", 9)

    for r in rows:
        if y < 80:
            c.showPage()
            y = height - margin
        vals = [str(r[0]), str(r[1])[:28], str(r[2])[:18], str(r[3]), f"{r[4]:.2f}", str(r[5])[:12], str(r[6])[:19]]
        for i, v in enumerate(vals):
            c.drawString(col_x[i], y, v)
        y -= 14

    c.save()
    messagebox.showinfo("PDF Created", f"Report saved to: {os.path.abspath(fname)}")
    if parent_window:
        parent_window.lift()

# ---------- GUI ----------
root = Tk()
root.title("Furniture Inventory System")
root.geometry("1000x650")
root.configure(bg="#EDE6DA")

title_label = Label(root, text="Furniture Inventory", font=("Arial", 22, "bold"),
                    bg="#EDE6DA", fg="#4B3A2F")
title_label.place(x=340, y=10)

form_frame = Frame(root, bg="#EDE6DA", bd=0)
form_frame.place(x=20, y=70, width=420, height=260)

Label(form_frame, text="Item Name", bg="#EDE6DA", fg="#4B3A2F").place(x=10, y=10)
name_entry = Entry(form_frame, width=28)
name_entry.place(x=120, y=10)

Label(form_frame, text="Category", bg="#EDE6DA", fg="#4B3A2F").place(x=10, y=50)
categories = ["Chair", "Table", "Bed", "Sofa", "Cupboard", "Accessory", "Other"]
category_var = StringVar(value=categories[0])
category_combo = ttk.Combobox(form_frame, textvariable=category_var, values=categories, width=26)
category_combo.place(x=120, y=50)

Label(form_frame, text="Quantity", bg="#EDE6DA", fg="#4B3A2F").place(x=10, y=90)
qty_entry = Entry(form_frame, width=28)
qty_entry.place(x=120, y=90)

Label(form_frame, text="Price", bg="#EDE6DA", fg="#4B3A2F").place(x=10, y=130)
price_entry = Entry(form_frame, width=28)
price_entry.place(x=120, y=130)

Label(form_frame, text="Country (Made In)", bg="#EDE6DA", fg="#4B3A2F").place(x=10, y=170)
country_entry = Entry(form_frame, width=28)
country_entry.place(x=120, y=170)

Button(form_frame, text="Add Item", width=12, bg="#A68A64", fg="white", command=add_item).place(x=10, y=210)
Button(form_frame, text="Update Item", width=12, bg="#A68A64", fg="white", command=update_item).place(x=130, y=210)
Button(form_frame, text="Delete Item", width=12, bg="#A68A64", fg="white", command=delete_item).place(x=250, y=210)

# ---------- SEARCH SECTION (FIXED) ----------
search_frame = Frame(root, bg="#EFEFEF", bd=0)
search_frame.place(x=460, y=70, width=500, height=120)

Label(search_frame, text="Search by ID", bg="#EFEFEF", font=("Arial", 10, "bold")).place(x=10, y=15)
search_id_entry = Entry(search_frame, width=20)
search_id_entry.place(x=140, y=15)

Label(search_frame, text="Search by Name", bg="#EFEFEF", font=("Arial", 10, "bold")).place(x=10, y=50)
search_name_entry = Entry(search_frame, width=25)
search_name_entry.place(x=140, y=50)

Button(search_frame, text="Search", bg="#8C6A43", fg="white", command=search_items).place(x=350, y=15)
Button(search_frame, text="Reset", bg="#6F5E53", fg="white", command=reset_search).place(x=350, y=50)
Button(search_frame, text="Show All", bg="#6F5E53", fg="white", command=lambda: show_items(None)).place(x=420, y=15)
Button(search_frame, text="Report", bg="#5A4B3D", fg="white", command=open_report_screen).place(x=420, y=50)

# Table
table_frame = Frame(root, bg="#EDE6DA")
table_frame.place(x=20, y=350, width=940, height=280)

style = ttk.Style()
style.configure("Treeview", background="#F7F3EE", foreground="black", rowheight=26, fieldbackground="#F7F3EE")
style.map("Treeview", background=[("selected", "#A68A64")])

tree = ttk.Treeview(table_frame, columns=("ID", "Name", "Category", "Qty", "Price", "Country", "Created"), show="headings")
for col, text in [("ID","ID"),("Name","Name"),("Category","Category"),("Qty","Qty"),("Price","Price"),("Country","Country"),("Created","Created At")]:
    tree.heading(col, text=text)
    tree.column(col, anchor=W, width=110 if col!="Name" else 200)

tree.pack(fill=BOTH, expand=True)
tree.bind("<ButtonRelease-1>", fill_entries)

show_items()
root.mainloop()